#  BƯỚC 0: GHÉP DỮ LIỆU (DATA MERGING)

## Tại sao cần ghép dữ liệu?

| File | Key liên kết | Cấu trúc |
|------|-------------|----------|
| `vietnam_destinations_normalized.csv` | date | 180 rows × 967 columns (wide format) |
| `vietnam_weather_monthly_2011_2025.csv` | province + date | 6,078 rows (long format) |
| `keyword_mapping_normalized.csv` | destination → province | Mapping table |

**Vấn đề**: Traffic data theo destination, Weather data theo province → Cần mapping để ghép

In [ ]:
# =============================================================================
# STEP 0.1: LOAD TẤT CẢ DỮ LIỆU
# =============================================================================
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(" Loading all data files...")

# Main traffic data (wide format: rows=months, cols=destinations)
traffic_df = pd.read_csv('../data/normalized/vietnam_destinations_normalized.csv')

# Weather data (long format: rows=province+month)
weather_df = pd.read_csv('../data/normalized/vietnam_weather_monthly_2011_2025.csv')

# Mapping destination → province
mapping_df = pd.read_csv('../data/normalized/keyword_mapping_normalized.csv')

# Festivals data
festivals_df = pd.read_csv('../data/normalized/vietnam_festivals.csv')

# Seasonal patterns (117 destinations với strong seasonality)
seasonal_df = pd.read_csv('../data/normalized/vietnam_seasonal_destinations_strong.csv')

# Destinations statistics
stats_df = pd.read_csv('../data/normalized/destinations_statistics.csv')

print("Traffic data: {} (rows=months, cols=destinations)".format(traffic_df.shape))
print(" Weather data: {}".format(weather_df.shape))
print(" Mapping data: {}".format(mapping_df.shape))
print(" Festivals data: {}".format(festivals_df.shape))
print(" Seasonal patterns: {}".format(seasonal_df.shape))
print(" Statistics: {}".format(stats_df.shape))


In [ ]:
# =============================================================================
# STEP 0.2: CHUYỂN ĐỔI DATE FORMAT
# =============================================================================

def parse_vietnamese_date(date_str):
    """Chuyển 'thg 1 2011' → datetime"""
    months_vi = {
        'thg 1': 1, 'thg 2': 2, 'thg 3': 3, 'thg 4': 4,
        'thg 5': 5, 'thg 6': 6, 'thg 7': 7, 'thg 8': 8,
        'thg 9': 9, 'thg 10': 10, 'thg 11': 11, 'thg 12': 12
    }
    parts = date_str.rsplit(' ', 1)  # Split from right: ['thg 1', '2011']
    month_str, year = parts[0], int(parts[1])
    month = months_vi.get(month_str, 1)
    return pd.Timestamp(year=year, month=month, day=1)

# Convert traffic date column
traffic_df['date_parsed'] = traffic_df['date'].apply(parse_vietnamese_date)
print(" Date range: {} to {}".format(traffic_df['date_parsed'].min(), traffic_df['date_parsed'].max()))
print("   Total months: {}".format(len(traffic_df)))


In [ ]:
# =============================================================================
# STEP 0.3: CHUYỂN TRAFFIC DATA TỪ WIDE → LONG FORMAT
# =============================================================================
# Wide format: rows=months, cols=destinations
# Long format: rows=month×destination (để dễ merge với weather)

# Get destination columns (all except 'date' and 'date_parsed')
destination_cols = [col for col in traffic_df.columns if col not in ['date', 'date_parsed']]
print(" Number of destinations: {}".format(len(destination_cols)))

# Melt to long format
traffic_long = traffic_df.melt(
    id_vars=['date_parsed'],
    value_vars=destination_cols,
    var_name='destination',
    value_name='traffic'
)

print("Traffic data after melt:")
print("   Shape: {}".format(traffic_long.shape))
print("   Columns: {}".format(traffic_long.columns.tolist()))
traffic_long.head()

In [ ]:
# =============================================================================
# STEP 0.4: GHÉP MAPPING (destination → province)
# =============================================================================

# Tạo mapping dict từ normalized_name → province_normalized
dest_to_province = dict(zip(
    mapping_df['normalized_name'], 
    mapping_df['province_normalized']
))

# Map province vào traffic_long
traffic_long['province'] = traffic_long['destination'].map(dest_to_province)

# Kiểm tra missing mappings
missing_mapping = traffic_long[traffic_long['province'].isna()]['destination'].unique()
print("️ Destinations without province mapping: {}".format(len(missing_mapping)))
if len(missing_mapping) > 0:
    print("   Examples: {}".format(missing_mapping[:5]))

# Loại bỏ rows không có province mapping
traffic_long_clean = traffic_long.dropna(subset=['province'])
print("Traffic data after mapping: {}".format(traffic_long_clean.shape))


In [ ]:
# =============================================================================
# STEP 0.5: GHÉP WEATHER DATA
# =============================================================================

# Convert weather date to same format
weather_df['date_parsed'] = pd.to_datetime(weather_df['date'])

# Merge traffic với weather theo province + date
merged_df = traffic_long_clean.merge(
    weather_df[['province', 'temp_avg', 'rainfall', 'date_parsed']],
    left_on=['province', 'date_parsed'],
    right_on=['province', 'date_parsed'],
    how='left'
)

print(" Merged data shape: {}".format(merged_df.shape))
print("   Missing weather data: {} rows".format(merged_df['temp_avg'].isna().sum()))

# Check sample
merged_df.head(10)

In [ ]:
# =============================================================================
# STEP 0.6: THÊM THÔNG TIN SEASONALITY
# =============================================================================

# Column names in seasonal_df are different - check and use correct names
print("Seasonal columns: {}".format(seasonal_df.columns.tolist()))

# Merge với seasonal patterns (using correct column names)
seasonal_info = seasonal_df[['Destination', 'Amplitude (Median Peak/Trough)', 'Peak_Months', 'Primary_Peak_Month']]
seasonal_info = seasonal_info.rename(columns={
    'Destination': 'destination',
    'Amplitude (Median Peak/Trough)': 'amplitude'
})

merged_df = merged_df.merge(seasonal_info, on='destination', how='left')

# Mark if destination has strong seasonality
merged_df['has_strong_seasonality'] = merged_df['amplitude'].notna()

print(" Destinations with strong seasonality: {}%".format(merged_df['has_strong_seasonality'].sum() / len(merged_df) * 100))

# Add time features
merged_df['year'] = merged_df['date_parsed'].dt.year
merged_df['month'] = merged_df['date_parsed'].dt.month
merged_df['quarter'] = merged_df['date_parsed'].dt.quarter

print("Final merged dataset:")
print("   Shape: {}".format(merged_df.shape))
print("   Columns: {}".format(merged_df.columns.tolist()))


In [ ]:
# =============================================================================
# STEP 0.7: LƯU DỮ LIỆU ĐÃ GHÉP
# =============================================================================

# Lưu merged dataset
output_path = '../data/normalized/merged_tourism_data.csv'
merged_df.to_csv(output_path, index=False)
print(" Saved merged data to: {}".format(output_path))

# Tóm tắt dữ liệu
print("\n" + "=" * 60)
print(" TÓM TẮT DỮ LIỆU ĐÃ GHÉP")
print("=" * 60)
print(" Total rows: {}".format(len(merged_df)))
print(" Unique destinations: {}".format(merged_df["destination"].nunique()))
print(" Unique provinces: {}".format(merged_df["province"].nunique()))
print(" Date range: {} to {}".format(merged_df["date_parsed"].min(), merged_df["date_parsed"].max()))
print(" Weather features: temp_avg, rainfall")
print(" Seasonality info: Amplitude, Peak_Months, Primary_Peak_Month")
print("=" * 60)

# Show sample
merged_df.sample(5)


##  GHÉP THÊM DỮ LIỆU BỔ SUNG

Các file còn lại cần ghép:
- **Economic**: GRDP, Population, Area
- **Infrastructure**: Accommodation, Restaurant, Entertainment, Healthcare, Shop counts
- **Geographic**: Regions, Distances
- **Social Media**: YouTube views/likes
- **Statistics**: Destination statistics

In [ ]:
# =============================================================================
# STEP 0.8: LOAD TẤT CẢ DỮ LIỆU BỔ SUNG
# =============================================================================

# Economic data
grdp_df = pd.read_csv('../data/normalized/vietnam_grdp_by_province.csv')
area_pop_df = pd.read_csv('../data/normalized/vietnam_area_population.csv')

# Geographic data  
regions_df = pd.read_csv('../data/normalized/vietnam_regions_with_distances.csv')

# Social media
youtube_df = pd.read_csv('../data/normalized/vietnam_youtube_province_aggregates.csv')

# Infrastructure counts (need to aggregate by province)
accommodation_df = pd.read_csv('../data/normalized/vietnam_accommodation.csv', sep=';')
restaurant_df = pd.read_csv('../data/normalized/vietnam_restaurants.csv', sep=';')
entertainment_df = pd.read_csv('../data/normalized/vietnam_entertainment.csv', sep=';')
healthcare_df = pd.read_csv('../data/normalized/vietnam_healthcare.csv', sep=';')
shops_df = pd.read_csv('../data/normalized/vietnam_shops.csv', sep=';')

# Destination statistics
dest_stats_df = pd.read_csv('../data/normalized/destinations_statistics.csv')

print(" Loaded additional data files:")
print("   GRDP: {}".format(grdp_df.shape))
print("   Area/Population: {}".format(area_pop_df.shape))
print("   Regions: {}".format(regions_df.shape))
print("   YouTube: {}".format(youtube_df.shape))
print("   Accommodation: {}".format(accommodation_df.shape))
print("   Restaurant: {}".format(restaurant_df.shape))
print("   Entertainment: {}".format(entertainment_df.shape))
print("   Healthcare: {}".format(healthcare_df.shape))
print("   Shops: {}".format(shops_df.shape))
print("   Dest Stats: {}".format(dest_stats_df.shape))


In [ ]:
# =============================================================================
# STEP 0.9: TẠO INFRASTRUCTURE COUNTS BY PROVINCE
# =============================================================================

# Count infrastructure by province
infra_counts = pd.DataFrame()

# Accommodation count
acc_count = accommodation_df.groupby('province_normalized').size().reset_index(name='accommodation_count')

# Restaurant count
rest_count = restaurant_df.groupby('province_normalized').size().reset_index(name='restaurant_count')

# Entertainment count
ent_count = entertainment_df.groupby('province_normalized').size().reset_index(name='entertainment_count')

# Healthcare count
health_count = healthcare_df.groupby('province_normalized').size().reset_index(name='healthcare_count')

# Shop count
shop_count = shops_df.groupby('province_normalized').size().reset_index(name='shop_count')

# Merge all counts
infra_counts = acc_count.merge(rest_count, on='province_normalized', how='outer')
infra_counts = infra_counts.merge(ent_count, on='province_normalized', how='outer')
infra_counts = infra_counts.merge(health_count, on='province_normalized', how='outer')
infra_counts = infra_counts.merge(shop_count, on='province_normalized', how='outer')
infra_counts = infra_counts.fillna(0)

# Rename column
infra_counts = infra_counts.rename(columns={'province_normalized': 'province'})

print(" Infrastructure counts by province: {}".format(infra_counts.shape))
infra_counts.head()

In [ ]:
# =============================================================================
# STEP 0.10: GHÉP REGIONS & GEOGRAPHIC DATA
# =============================================================================

# Prepare regions data (static by province)
regions_clean = regions_df[['province_normalized', 'region', 'latitude', 'longitude', 
                            'distance_to_hanoi_km', 'distance_to_hcm_km']].copy()
regions_clean = regions_clean.rename(columns={'province_normalized': 'province'})

# Merge regions into main dataset
merged_df = merged_df.merge(regions_clean, on='province', how='left')

print(" After merging regions: {}".format(merged_df.shape))
print("   Columns: {}".format(merged_df.columns.tolist()))


In [ ]:
# =============================================================================
# STEP 0.11: GHÉP INFRASTRUCTURE COUNTS
# =============================================================================

# Merge infrastructure counts into main dataset
merged_df = merged_df.merge(infra_counts, on='province', how='left')

# Fill NaN with 0 for infrastructure counts
infra_cols = ['accommodation_count', 'restaurant_count', 'entertainment_count', 
              'healthcare_count', 'shop_count']
merged_df[infra_cols] = merged_df[infra_cols].fillna(0)

print(" After merging infrastructure: {}".format(merged_df.shape))


In [ ]:
# =============================================================================
# STEP 0.12: GHÉP YOUTUBE DATA
# =============================================================================

# Prepare YouTube data
youtube_clean = youtube_df[['province_normalized', 'views', 'likes', 'comments']].copy()
youtube_clean = youtube_clean.rename(columns={
    'province_normalized': 'province',
    'views': 'youtube_views',
    'likes': 'youtube_likes',
    'comments': 'youtube_comments'
})

# Merge YouTube data
merged_df = merged_df.merge(youtube_clean, on='province', how='left')

print(" After merging YouTube: {}".format(merged_df.shape))


In [ ]:
# =============================================================================
# STEP 0.13: GHÉP GRDP DATA (theo province + year)
# =============================================================================

# Check actual column names
print("GRDP columns: {}".format(grdp_df.columns.tolist()))

# Clean GRDP data - find the GRDP column
grdp_col = [col for col in grdp_df.columns if 'GRDP' in col][0]
grdp_clean = grdp_df[['Năm', 'province_normalized', grdp_col]].copy()
grdp_clean.columns = ['year', 'province', 'grdp']

# Clean GRDP values (remove dots and commas, convert to float)
grdp_clean['grdp'] = grdp_clean['grdp'].astype(str).str.replace('.', '', regex=False)
grdp_clean['grdp'] = grdp_clean['grdp'].str.replace(',', '.', regex=False)
grdp_clean['grdp'] = pd.to_numeric(grdp_clean['grdp'], errors='coerce')

# Merge GRDP data by province + year
merged_df = merged_df.merge(grdp_clean, on=['province', 'year'], how='left')

print(" After merging GRDP: {}".format(merged_df.shape))
print("   GRDP coverage: {}%".format(merged_df['grdp'].notna().sum() / len(merged_df) * 100))


In [ ]:
# =============================================================================
# STEP 0.14: GHÉP AREA & POPULATION DATA (theo province + year)
# =============================================================================

# Clean area/population data
area_pop_clean = area_pop_df[['Năm', 'province_normalized', 'Diện tích (Km2)', 
                              'Dân số trung bình (nghìn)', 'Mật độ dân số (người/km2)']].copy()
area_pop_clean.columns = ['year', 'province', 'area_km2', 'population_thousand', 'density']

# Merge area/population data by province + year
merged_df = merged_df.merge(area_pop_clean, on=['province', 'year'], how='left')

print(" After merging area/population: {}".format(merged_df.shape))
print("   Population coverage: {}%".format(merged_df['population_thousand'].notna().sum() / len(merged_df) * 100))


In [ ]:
# =============================================================================
# STEP 0.15: GHÉP DESTINATION STATISTICS & LOẠI BỎ DUPLICATES
# =============================================================================

# Loại bỏ duplicates nếu có
print("Before removing duplicates: {}".format(merged_df.shape))
merged_df = merged_df.drop_duplicates(subset=['date_parsed', 'destination'])
print("After removing duplicates: {}".format(merged_df.shape))

# Clean destination statistics
dest_stats_clean = dest_stats_df.rename(columns={
    'Destination': 'destination',
    'Mean': 'dest_mean_traffic',
    'Median': 'dest_median_traffic', 
    'Max': 'dest_max_traffic',
    'Std Dev': 'dest_std_traffic',
    'Coverage %': 'dest_coverage_pct'
})

# Merge destination statistics
merged_df = merged_df.merge(
    dest_stats_clean[['destination', 'dest_mean_traffic', 'dest_median_traffic', 
                      'dest_max_traffic', 'dest_std_traffic', 'dest_coverage_pct']], 
    on='destination', 
    how='left'
)

print(" After merging destination stats: {}".format(merged_df.shape))


In [ ]:
# =============================================================================
# STEP 0.16: TÓM TẮT VÀ LƯU DỮ LIỆU HOÀN CHỈNH
# =============================================================================

# Final summary
print("=" * 70)
print(" TÓM TẮT DỮ LIỆU ĐÃ GHÉP HOÀN CHỈNH")
print("=" * 70)
print("Dataset shape: {}".format(merged_df.shape))
print(" Unique destinations: {}".format(merged_df['destination'].nunique()))
print("️ Unique provinces: {}".format(merged_df['province'].nunique()))
print(" Date range: {} to {}".format(merged_df['date_parsed'].min(), merged_df['date_parsed'].max()))

print("ALL COLUMNS ({}):".format(len(merged_df.columns)))
for i, col in enumerate(merged_df.columns):
    print("   {}. {}".format(i + 1, col))

# Save complete merged dataset
output_path_full = '../data/normalized/merged_tourism_data_complete.csv'
merged_df.to_csv(output_path_full, index=False)
print("Saved complete merged data to: {}".format(output_path_full))


In [ ]:
# =============================================================================
# STEP 0.17: KIỂM TRA MISSING VALUES
# =============================================================================

# Check missing values
missing_summary = merged_df.isnull().sum()
missing_pct = (missing_summary / len(merged_df) * 100).round(2)

missing_df = pd.DataFrame({
    'Column': missing_summary.index,
    'Missing Count': missing_summary.values,
    'Missing %': missing_pct.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing %', ascending=False)

print("️ MISSING VALUES SUMMARY:")
print(missing_df.to_string(index=False))

# Show sample of final dataset
print("\n SAMPLE DATA:")
merged_df.sample(3)

---
#  HOÀN TẤT GHÉP DỮ LIỆU

Bây giờ bạn có thể sử dụng `merged_df` hoặc load từ file `merged_tourism_data.csv` cho các phân tích tiếp theo.

**Cấu trúc dữ liệu đã ghép:**
| Column | Description |
|--------|-------------|
| date_parsed | Ngày (monthly) |
| destination | Tên điểm đến |
| traffic | Lượng tìm kiếm |
| province | Tỉnh/TP |
| temp_avg | Nhiệt độ trung bình |
| rainfall | Lượng mưa |
| Amplitude | Biên độ dao động (seasonality) |
| Peak_Months | Tháng cao điểm |
| year, month, quarter | Time features |

---

### 1️⃣ DỮ LIỆU CHÍNH (Core Data)

| File | Mô tả | Số dòng | Columns chính |
|------|-------|---------|---------------|
| `vietnam_destinations_normalized.csv` | **Traffic tìm kiếm theo tháng** cho 967 địa điểm (2011-2025) | 180 rows | date, [967 destination columns] |
| `destinations_statistics.csv` | Thống kê tổng hợp cho mỗi địa điểm | 968 rows | Destination, Mean, Median, Max, Min, Std Dev, Non-zero Count, Coverage % |
| `keyword_mapping_normalized.csv` | Mapping địa điểm → tỉnh/thành | 967 rows | original_name, normalized_name, province, province_normalized |
| `vietnam_seasonal_destinations_strong.csv` | **Địa điểm có tính mùa vụ mạnh** | 117 rows | Destination, Peak_Months, Primary_Peak_Month, Amplitude, CV |

### 2️⃣ DỮ LIỆU BỔ TRỢ (Supporting Data)

| File | Mô tả | Dùng cho |
|------|-------|----------|
| `vietnam_weather_monthly_2011_2025.csv` | Thời tiết theo tháng (nhiệt độ, lượng mưa) | Feature thời tiết |
| `vietnam_festivals.csv` | Lễ hội theo tháng (2018-2025) | Feature lễ hội |
| `vietnam_youtube_province_aggregates.csv` | Views/Likes YouTube theo tỉnh | Feature social media |
| `vietnam_grdp_by_province.csv` | GRDP theo tỉnh | Feature kinh tế |
| `vietnam_population.csv` | Dân số theo tỉnh | Feature dân số |
| `vietnam_regions_with_distances.csv` | Khoảng cách từ HN/HCM | Feature địa lý |
| `vietnam_accommodation.csv` | Số lượng khách sạn/nhà nghỉ | Feature hạ tầng |
| `vietnam_restaurants.csv` | Số lượng nhà hàng | Feature hạ tầng |
| `vietnam_entertainment.csv` | Điểm vui chơi giải trí | Feature hạ tầng |

---
##  PHẦN 1: KHÁM PHÁ DỮ LIỆU (EDA)

In [ ]:
# Import thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Cài đặt hiển thị
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12
pd.set_option('display.max_columns', None)

# Đường dẫn dữ liệu
DATA_PATH = '../data/normalized/'

In [ ]:
# 1. Load dữ liệu chính - Traffic tìm kiếm
df_traffic = pd.read_csv(DATA_PATH + 'vietnam_destinations_normalized.csv')

# Chuyển đổi cột date
# Format: 'thg X YYYY' -> datetime
def parse_vietnamese_date(date_str):
    """Chuyển đổi format 'thg 1 2011' thành datetime"""
    parts = date_str.split()
    month = int(parts[1])
    year = int(parts[2])
    return pd.Timestamp(year=year, month=month, day=1)

df_traffic['date'] = df_traffic['date'].apply(parse_vietnamese_date)
df_traffic = df_traffic.set_index('date')

print(" Khoảng thời gian: {} đến {}".format(df_traffic.index.min(), df_traffic.index.max()))
print(" Số địa điểm: {}".format(len(df_traffic.columns)))
print(" Số tháng dữ liệu: {}".format(len(df_traffic)))


In [ ]:
# 2. Load thống kê địa điểm
df_stats = pd.read_csv(DATA_PATH + 'destinations_statistics.csv')
print("\n Phân phối Mean Traffic:")
print(df_stats['Mean'].describe())

# Phân loại địa điểm theo mức traffic
df_stats['traffic_level'] = pd.cut(
    df_stats['Mean'], 
    bins=[0, 1, 10, 50, 100, float('inf')],
    labels=['Rất thấp', 'Thấp', 'Trung bình', 'Cao', 'Rất cao']
)
print("\n Phân loại địa điểm theo traffic:")
print(df_stats['traffic_level'].value_counts())

In [ ]:
# 3. Load dữ liệu mùa vụ
df_seasonal = pd.read_csv(DATA_PATH + 'vietnam_seasonal_destinations_strong.csv')
print("Số địa điểm có tính mùa vụ mạnh: {}".format(len(df_seasonal)))

# Top 10 địa điểm có tính mùa vụ mạnh nhất
print("\n Top 10 địa điểm có biên độ mùa vụ cao nhất:")
df_seasonal.nlargest(10, 'Amplitude (Median Peak/Trough)')[['Destination', 'Peak_Months', 'Amplitude (Median Peak/Trough)', 'Primary_Peak_Month']]

In [ ]:
# 4. Mapping địa điểm với tỉnh/thành
df_mapping = pd.read_csv(DATA_PATH + 'keyword_mapping_normalized.csv')
print("\n Phân bố địa điểm theo tỉnh/thành (Top 15):")
print(df_mapping['province_normalized'].value_counts().head(15))

---
##  PHẦN 2: PHÂN TÍCH XU HƯỚNG

In [ ]:
# 2.1 Phân tích xu hướng tổng thể
# Tính tổng traffic theo tháng
total_traffic = df_traffic.sum(axis=1)

fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Plot 1: Xu hướng tổng
axes[0].plot(total_traffic.index, total_traffic.values, linewidth=2, color='blue')
axes[0].set_title(' Xu hướng Traffic Tìm Kiếm Du Lịch Tổng Thể (2011-2025)', fontsize=14)
axes[0].set_xlabel('Năm')
axes[0].set_ylabel('Tổng Traffic')
axes[0].axhline(y=total_traffic.mean(), color='red', linestyle='--', label=f'Trung bình: {total_traffic.mean():.0f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Phân tích theo tháng
monthly_avg = df_traffic.groupby(df_traffic.index.month).mean().mean(axis=1)
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, 12))
bars = axes[1].bar(range(1, 13), monthly_avg.values, color=colors)
axes[1].set_title(' Traffic Trung Bình Theo Tháng', fontsize=14)
axes[1].set_xlabel('Tháng')
axes[1].set_ylabel('Traffic Trung Bình')
axes[1].set_xticks(range(1, 13))
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# 2.2 Phân tích YoY Growth (Year-over-Year)
# Tính growth rate năm sau so với năm trước
yearly_traffic = df_traffic.groupby(df_traffic.index.year).sum().sum(axis=1)
yearly_growth = yearly_traffic.pct_change() * 100

print(" Tăng trưởng Traffic YoY (%):")
for year, growth in yearly_growth.dropna().items():
    emoji = "" if growth > 0 else ""
    print("{} {}: {}%".format(emoji, year, growth))


---
##  PHẦN 3: FEATURE ENGINEERING

In [ ]:
# 3.1 Tạo features từ dữ liệu thời tiết
df_weather = pd.read_csv(DATA_PATH + 'vietnam_weather_monthly_2011_2025.csv')
df_weather['date'] = pd.to_datetime(df_weather['date'])

# Tạo features thời tiết theo tỉnh và tháng
weather_features = df_weather.groupby(['province', df_weather['date'].dt.to_period('M')]).agg({
    'temp_avg': 'mean',
    'rainfall': 'sum'
}).reset_index()

print(" Features Thời Tiết:")
print(weather_features.head())

In [ ]:
# 3.2 Tạo features từ lễ hội
df_festivals = pd.read_csv(DATA_PATH + 'vietnam_festivals.csv')

# Đếm số lễ hội theo tỉnh và tháng
festival_columns = [col for col in df_festivals.columns if col.startswith('month_gregorian')]

# Chuyển đổi thành format dài
festival_long = df_festivals.melt(
    id_vars=['province', 'festival_name', 'type', 'province_normalized'],
    value_vars=festival_columns,
    var_name='year_col',
    value_name='month'
)

# Đếm số lễ hội
festival_long['month'] = pd.to_datetime(festival_long['month'])
festival_count = festival_long.groupby(['province_normalized', festival_long['month'].dt.to_period('M')]).size().reset_index(name='festival_count')
print(" Số lễ hội theo tỉnh và tháng:")
print(festival_count.head(10))


In [ ]:
# 3.3 Tạo features lag và rolling
def create_time_features(series, destination_name):
    """Tạo features thời gian cho một địa điểm"""
    df = pd.DataFrame()
    df['traffic'] = series
    
    # Lag features (tháng trước)
    for i in [1, 2, 3, 6, 12]:
        df[f'lag_{i}m'] = series.shift(i)
    
    # Rolling statistics
    df['rolling_mean_3m'] = series.rolling(window=3).mean()
    df['rolling_mean_6m'] = series.rolling(window=6).mean()
    df['rolling_std_3m'] = series.rolling(window=3).std()
    
    # Growth rate
    df['mom_growth'] = series.pct_change()  # Month-over-Month
    df['yoy_growth'] = series.pct_change(12)  # Year-over-Year
    
    # Seasonal features
    df['month'] = series.index.month
    df['quarter'] = series.index.quarter
    df['year'] = series.index.year
    
    # Cyclical encoding cho month
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    return df

# Demo với một địa điểm
sample_dest = 'Hồ Hoàn Kiếm'
if sample_dest in df_traffic.columns:
    sample_features = create_time_features(df_traffic[sample_dest], sample_dest)
    print(" Features cho {}:".format(sample_dest))
    print(sample_features.tail(10))

---
##  PHẦN 4: XÂY DỰNG MÔ HÌNH DỰ ĐOÁN

###  CHIẾN LƯỢC MÔ HÌNH

#### A. Các Mô Hình Đề Xuất:

| Mô hình | Ưu điểm | Nhược điểm | Phù hợp khi |
|---------|---------|------------|-------------|
| **SARIMA** | Bắt được tính mùa vụ, interpretable | Chỉ dùng 1 chuỗi, linear | Dữ liệu có mùa vụ rõ ràng |
| **Prophet** | Dễ dùng, tự động detect seasonality | Cần tune nhiều | Time series với trend và mùa vụ |
| **XGBoost** | Xử lý nhiều features, robust | Cần feature engineering | Có nhiều features ngoại sinh |
| **LSTM** | Bắt được pattern phức tạp | Cần nhiều data, khó tune | Dữ liệu lớn, pattern phức tạp |
| **Ensemble** | Kết hợp ưu điểm các mô hình | Phức tạp | Production system |

#### B. Chiến Lược Dự Đoán Ngắn Hạn (1-3 tháng):

1. **Mô hình cấp địa điểm**: Dự đoán từng destination riêng biệt
2. **Mô hình cấp tỉnh**: Aggregate theo tỉnh rồi dự đoán
3. **Mô hình kết hợp**: Dự đoán tổng + phân bổ theo tỉ lệ lịch sử

In [ ]:
# 4.1 Cài đặt thêm thư viện (nếu cần)
# !pip install prophet statsmodels xgboost scikit-learn

In [ ]:
# 4.2 Mô hình SARIMA cho dự đoán ngắn hạn
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

def build_sarima_model(series, train_size=0.8):
    """
    Xây dựng mô hình SARIMA cho một chuỗi thời gian
    """
    # Chia train/test
    n = len(series)
    train = series[:int(n * train_size)]
    test = series[int(n * train_size):]
    
    # Fit model với SARIMA(1,1,1)(1,1,1,12) - seasonal period = 12 tháng
    try:
        model = SARIMAX(
            train, 
            order=(1, 1, 1),
            seasonal_order=(1, 1, 1, 12),
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        fitted = model.fit(disp=False)
        
        # Dự đoán
        predictions = fitted.forecast(len(test))
        
        # Đánh giá
        mae = mean_absolute_error(test, predictions)
        rmse = np.sqrt(mean_squared_error(test, predictions))
        
        return {
            'model': fitted,
            'predictions': predictions,
            'actual': test,
            'mae': mae,
            'rmse': rmse
        }
    except Exception as e:
        return {'error': str(e)}

# Demo với một địa điểm có traffic cao
high_traffic_destinations = df_stats.nlargest(10, 'Mean')['Destination'].tolist()
print(" Top 10 địa điểm có traffic cao nhất:")
for i, dest in enumerate(high_traffic_destinations, 1):
    mean_val = df_stats[df_stats['Destination'] == dest]['Mean'].values[0]
    print("  {}. {}: {}".format(i, dest, mean_val))


In [ ]:
# 4.3 XGBoost với features đầy đủ
from sklearn.model_selection import TimeSeriesSplit

def prepare_ml_dataset(df_traffic, df_mapping, destination):
    """
    Chuẩn bị dataset cho ML model
    """
    if destination not in df_traffic.columns:
        return None
    
    series = df_traffic[destination]
    df = create_time_features(series, destination)
    
    # Thêm thông tin tỉnh/thành
    province_info = df_mapping[df_mapping['normalized_name'] == destination]
    if len(province_info) > 0:
        province = province_info['province_normalized'].values[0]
        df['province'] = province
    
    # Drop NaN
    df = df.dropna()
    
    return df

print(" Functions đã sẵn sàng cho training model")

---
##  PHẦN 5: PHÂN TÍCH CHUYÊN SÂU CHO DỰ ĐOÁN NGẮN HẠN

In [ ]:
# 5.1 Xác định địa điểm có tiềm năng tăng trưởng trong kỳ tới
def identify_trending_destinations(df_traffic, df_seasonal, n_months=3):
    """
    Xác định địa điểm có khả năng có traffic cao trong n tháng tới
    dựa trên:
    1. Peak months của địa điểm đó
    2. Trend gần đây
    3. YoY comparison
    """
    current_month = datetime.now().month
    upcoming_months = [(current_month + i) % 12 or 12 for i in range(1, n_months + 1)]
    
    print("️ Phân tích cho {} tháng tới: {}".format(n_months, upcoming_months))
    
    potential_destinations = []
    
    for _, row in df_seasonal.iterrows():
        dest = row['Destination']
        peak_months = eval(row['Peak_Months_List']) if isinstance(row['Peak_Months_List'], str) else []
        
        # Check nếu peak months nằm trong upcoming months
        overlap = set(peak_months) & set(upcoming_months)
        if overlap:
            potential_destinations.append({
                'destination': dest,
                'peak_months': peak_months,
                'upcoming_peak_months': list(overlap),
                'amplitude': row['Amplitude (Median Peak/Trough)'],
                'primary_peak': row['Primary_Peak_Month']
            })
    
    result_df = pd.DataFrame(potential_destinations)
    return result_df.sort_values('amplitude', ascending=False)

# Chạy phân tích
trending = identify_trending_destinations(df_traffic, df_seasonal, n_months=3)
print("Số địa điểm có peak trong 3 tháng tới: {}".format(len(trending)))
print("\n Top 20 địa điểm tiềm năng:")
trending.head(20)

In [ ]:
# 5.2 Phân tích correlation với các yếu tố
def analyze_traffic_drivers(df_traffic, df_weather, df_mapping, destination):
    """
    Phân tích các yếu tố ảnh hưởng đến traffic của một địa điểm
    """
    if destination not in df_traffic.columns:
        return None
    
    # Lấy province
    province_info = df_mapping[df_mapping['normalized_name'] == destination]
    if len(province_info) == 0:
        return None
    
    province = province_info['province_normalized'].values[0]
    
    # Lấy traffic
    traffic = df_traffic[destination].reset_index()
    traffic.columns = ['date', 'traffic']
    
    # Merge với weather
    weather_subset = df_weather[df_weather['province'] == province].copy()
    if len(weather_subset) == 0:
        return None
    
    # Tính correlation
    traffic['month'] = traffic['date'].dt.month
    weather_subset['month'] = pd.to_datetime(weather_subset['date']).dt.month
    
    # Thống kê đơn giản
    result = {
        'destination': destination,
        'province': province,
        'avg_traffic': traffic['traffic'].mean(),
        'peak_month': traffic.groupby('month')['traffic'].mean().idxmax(),
        'low_month': traffic.groupby('month')['traffic'].mean().idxmin()
    }
    
    return result

print(" Function analyze_traffic_drivers đã sẵn sàng")

---
##  PHẦN 6: ĐỀ XUẤT & KẾT LUẬN

###  HƯỚNG PHÂN TÍCH CHI TIẾT

#### BƯỚC 1: Data Understanding & Cleaning
1.  Load và khám phá tất cả datasets
2.  Xử lý missing values
3.  Chuẩn hóa tên tỉnh/thành
4.  Merge các nguồn dữ liệu

#### BƯỚC 2: EDA Chuyên Sâu
1. Phân tích xu hướng tổng thể traffic
2. Phân tích mùa vụ theo tháng/quý
3. Phân tích theo vùng miền
4. Xác định địa điểm có tính mùa vụ mạnh
5. Phân tích correlation với thời tiết, lễ hội

#### BƯỚC 3: Feature Engineering
1. **Time features**: Lag, rolling mean/std, growth rates
2. **Seasonal features**: Month sin/cos, quarter, is_holiday
3. **External features**: Weather, festivals, GRDP
4. **Infrastructure features**: Số khách sạn, nhà hàng

#### BƯỚC 4: Modeling
1. **Baseline**: SARIMA cho từng địa điểm
2. **Advanced**: XGBoost/LightGBM với all features
3. **Ensemble**: Kết hợp nhiều mô hình
4. **Evaluation**: MAE, RMSE, MAPE

#### BƯỚC 5: Prediction & Deployment
1. Dự đoán traffic 1-3 tháng tới
2. Ranking địa điểm tiềm năng
3. Đề xuất chiến lược marketing
4. Dashboard monitoring

###  METRICS ĐÁNH GIÁ

| Metric | Công thức | Ý nghĩa |
|--------|-----------|----------|
| **MAE** | Mean Absolute Error | Sai số trung bình tuyệt đối |
| **RMSE** | Root Mean Squared Error | Penalize lỗi lớn nhiều hơn |
| **MAPE** | Mean Absolute Percentage Error | Sai số % - dễ hiểu |
| **Hit Rate** | % địa điểm predicted top-K nằm trong actual top-K | Đánh giá ranking |

###  ỨNG DỤNG THỰC TẾ

1. **Marketing Campaign**: Target địa điểm có traffic predicted cao
2. **Resource Planning**: Chuẩn bị hạ tầng, nhân lực
3. **Content Creation**: Tạo content cho địa điểm sắp peak
4. **Promotion Timing**: Chạy promotion trước khi peak

---
##  NEXT STEPS

1. **Hoàn thiện EDA**: Phân tích sâu hơn từng loại địa điểm
2. **Build models**: Implement SARIMA, Prophet, XGBoost
3. **Validate**: Cross-validation, backtesting
4. **Production**: API, Dashboard
5. **Monitor**: Theo dõi và cập nhật model

---

** Note**: Notebook này là template hướng dẫn. Cần chạy và tinh chỉnh theo dữ liệu thực tế.

---
#  PHẦN 7: SỬ DỤNG DATASET ĐÃ MERGE SẴN

## Ưu điểm của `merged_tourism_data_complete.csv`
- **174,060 dòng** (mỗi destination × mỗi tháng)
- **35 features** đã được tính toán sẵn
- Đã xử lý missing values và chuẩn hóa tên tỉnh

### Các cột quan trọng:
| Nhóm | Cột | Mô tả |
|------|-----|-------|
| **Target** | `traffic` | Số lượng tìm kiếm (để dự đoán) |
| **Time** | `date_parsed`, `year`, `month`, `quarter` | Thời gian |
| **Location** | `destination`, `province`, `region`, `latitude`, `longitude` | Vị trí |
| **Weather** | `temp_avg`, `rainfall` | Thời tiết tháng đó |
| **Seasonality** | `amplitude`, `Peak_Months`, `has_strong_seasonality` | Đặc điểm mùa vụ |
| **Infrastructure** | `accommodation_count`, `restaurant_count`, `entertainment_count`, `healthcare_count`, `shop_count` | Hạ tầng du lịch |
| **Social** | `youtube_views`, `youtube_likes`, `youtube_comments` | Tín hiệu social media |
| **Economy** | `grdp`, `area_km2`, `population_thousand`, `density` | Kinh tế tỉnh |
| **Distance** | `distance_to_hanoi_km`, `distance_to_hcm_km` | Khoảng cách đến 2 TP lớn |
| **Stats** | `dest_mean_traffic`, `dest_median_traffic`, `dest_max_traffic`, `dest_std_traffic`, `dest_coverage_pct` | Thống kê của destination |

In [ ]:
# =============================================================================
# BƯỚC 7.1: LOAD DATASET ĐÃ MERGE SẴN
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Load merged dataset
df = pd.read_csv('../data/normalized/merged_tourism_data_complete.csv')
df['date_parsed'] = pd.to_datetime(df['date_parsed'])

print(" DATASET OVERVIEW")
print("   Shape: {}".format(df.shape))
print("   Date range: {} → {}".format(df['date_parsed'].min(), df['date_parsed'].max()))
print("   Unique destinations: {}".format(df['destination'].nunique()))
print("   Unique provinces: {}".format(df['province'].nunique()))
print("Columns: {}".format(df.columns.tolist()))
print("Memory usage: {} MB".format(df.memory_usage(deep=True).sum() / 1024 ** 2))


In [ ]:
# =============================================================================
# BƯỚC 7.2: PHÂN TÍCH SƠ BỘ TARGET VARIABLE (TRAFFIC)
# =============================================================================

print(" TARGET VARIABLE: traffic\n")
print(df['traffic'].describe())

# Phân tích phân phối
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. Distribution (log scale vì highly skewed)
ax1 = axes[0]
df_nonzero = df[df['traffic'] > 0]['traffic']
ax1.hist(np.log1p(df_nonzero), bins=50, edgecolor='white')
ax1.set_xlabel('log1p(traffic)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Traffic (log scale)')

# 2. Zero-inflation analysis
ax2 = axes[1]
zero_pct = (df['traffic'] == 0).mean() * 100
nonzero_pct = 100 - zero_pct
ax2.bar(['Zero', 'Non-Zero'], [zero_pct, nonzero_pct], color=['#e74c3c', '#27ae60'])
ax2.set_ylabel('Percentage')
ax2.set_title(f'Zero-Inflation: {zero_pct:.1f}% là zero')

# 3. Traffic theo tháng
ax3 = axes[2]
monthly_traffic = df.groupby('month')['traffic'].mean()
ax3.bar(monthly_traffic.index, monthly_traffic.values, color='#3498db')
ax3.set_xlabel('Month')
ax3.set_ylabel('Average Traffic')
ax3.set_title('Seasonality: Traffic theo tháng')

plt.tight_layout()
plt.show()

print("
️ LƯU Ý: {}% dữ liệu = 0 → Cần xử lý zero-inflation".format(zero_pct))


## ️ BƯỚC 7.3: FEATURE ENGINEERING CHO DỰ ĐOÁN NGẮN HẠN

### Chiến lược Feature Engineering:

| Loại Feature | Tên | Công thức | Mục đích |
|--------------|-----|-----------|----------|
| **Lag Features** | `lag_1m`, `lag_2m`, `lag_3m` | traffic(t-1), traffic(t-2), traffic(t-3) | Xu hướng gần đây |
| **Rolling Features** | `rolling_3m_mean`, `rolling_3m_std` | mean/std của 3 tháng gần nhất | Trung bình động |
| **YoY Features** | `traffic_same_month_last_year` | traffic của cùng tháng năm trước | Tính mùa vụ năm |
| **Growth Rate** | `growth_1m`, `growth_yoy` | (t - t-1) / t-1 | Tốc độ tăng trưởng |
| **Seasonal** | `month_sin`, `month_cos` | sin/cos của tháng | Seasonal encoding |
| **Is Peak** | `is_peak_month` | Tháng hiện tại có trong Peak_Months không | Binary flag |
| **Weather Delta** | `temp_anomaly` | temp_avg - temp trung bình tháng đó | Thời tiết bất thường |

In [ ]:
# =============================================================================
# BƯỚC 7.3: TẠO FEATURES CHO MODELING
# =============================================================================

def create_features(df):
    """
    Tạo features cho dự đoán traffic ngắn hạn
    """
    df = df.sort_values(['destination', 'date_parsed']).copy()
    
    # 1. LAG FEATURES - Quan trọng nhất cho dự đoán ngắn hạn
    for lag in [1, 2, 3, 6, 12]:
        df[f'lag_{lag}m'] = df.groupby('destination')['traffic'].shift(lag)
    
    # 2. ROLLING FEATURES - Trung bình động
    df['rolling_3m_mean'] = df.groupby('destination')['traffic'].transform(
        lambda x: x.shift(1).rolling(3, min_periods=1).mean()
    )
    df['rolling_3m_std'] = df.groupby('destination')['traffic'].transform(
        lambda x: x.shift(1).rolling(3, min_periods=1).std()
    )
    df['rolling_6m_mean'] = df.groupby('destination')['traffic'].transform(
        lambda x: x.shift(1).rolling(6, min_periods=1).mean()
    )
    
    # 3. GROWTH RATE
    df['growth_1m'] = (df['traffic'] - df['lag_1m']) / (df['lag_1m'] + 1)  # +1 để tránh chia 0
    df['growth_yoy'] = (df['traffic'] - df['lag_12m']) / (df['lag_12m'] + 1)
    
    # 4. SEASONAL ENCODING (cyclical)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    # 5. IS PEAK MONTH (dựa trên Peak_Months column)
    def check_peak(row):
        if pd.isna(row['Peak_Months']):
            return 0
        try:
            peak_months = [int(x.strip()) for x in str(row['Peak_Months']).split(',')]
            return 1 if row['month'] in peak_months else 0
        except:
            return 0
    
    df['is_peak_month'] = df.apply(check_peak, axis=1)
    
    # 6. WEATHER ANOMALY (so với trung bình tháng đó của tỉnh)
    monthly_avg_temp = df.groupby(['province', 'month'])['temp_avg'].transform('mean')
    df['temp_anomaly'] = df['temp_avg'] - monthly_avg_temp
    
    # 7. RELATIVE TRAFFIC (so với destination mean)
    df['relative_traffic'] = df['traffic'] / (df['dest_mean_traffic'] + 1)
    
    # 8. YEAR TREND
    df['year_trend'] = df['year'] - df['year'].min()
    
    return df

# Apply feature engineering
print(" Creating features...")
df_features = create_features(df)
print(" Shape after feature engineering: {}".format(df_features.shape))
print(" New columns: {}".format([c for c in df_features.columns if c not in df.columns]))


##  BƯỚC 7.4: TRAIN/TEST SPLIT CHO TIME SERIES

### ️ LƯU Ý QUAN TRỌNG:
- **KHÔNG được dùng random split** cho time series (data leakage)
- Phải chia theo thời gian: train trên quá khứ, test trên tương lai
- Sử dụng **Rolling Origin Cross-Validation** để đánh giá

In [ ]:
# =============================================================================
# BƯỚC 7.4: CHIA DỮ LIỆU TRAIN/VALIDATION/TEST
# =============================================================================

# Time-based split
# Train: 2011-2022, Validation: 2023, Test: 2024-2025
train_df = df_features[df_features['year'] <= 2022].copy()
val_df = df_features[df_features['year'] == 2023].copy()
test_df = df_features[df_features['year'] >= 2024].copy()

print(" DATA SPLIT (Time-based):")
print("   Train:      {} rows ({}-{})".format(train_df.shape[0], train_df['year'].min(), train_df['year'].max()))
print("   Validation: {} rows ({}-{})".format(val_df.shape[0], val_df['year'].min(), val_df['year'].max()))
print("   Test:       {} rows ({}-{})".format(test_df.shape[0], test_df['year'].min(), test_df['year'].max()))

# Loại bỏ các dòng có missing values do lag features (các tháng đầu tiên)
print("Missing values due to lag features:")
print("   Train before dropna: {}".format(len(train_df)))
train_df = train_df.dropna(subset=['lag_1m', 'lag_12m'])
print("   Train after dropna: {}".format(len(train_df)))


In [ ]:
# =============================================================================
# BƯỚC 7.5: ĐỊNH NGHĨA FEATURES CHO MODEL
# =============================================================================

# Features to use
FEATURE_COLS = [
    # Lag features (quan trọng nhất)
    'lag_1m', 'lag_2m', 'lag_3m', 'lag_6m', 'lag_12m',
    
    # Rolling statistics
    'rolling_3m_mean', 'rolling_3m_std', 'rolling_6m_mean',
    
    # Time features
    'month_sin', 'month_cos', 'quarter', 'year_trend',
    
    # Seasonality
    'is_peak_month', 'has_strong_seasonality', 'amplitude',
    
    # Weather
    'temp_avg', 'rainfall', 'temp_anomaly',
    
    # Infrastructure
    'accommodation_count', 'restaurant_count', 'entertainment_count', 
    'shop_count', 'healthcare_count',
    
    # Location
    'latitude', 'longitude', 'distance_to_hanoi_km', 'distance_to_hcm_km',
    
    # Social
    'youtube_views', 'youtube_likes',
    
    # Economics
    'grdp', 'population_thousand', 'density',
    
    # Destination stats
    'dest_mean_traffic', 'dest_max_traffic', 'dest_coverage_pct'
]

TARGET = 'traffic'

# Filter only columns that exist
FEATURE_COLS = [c for c in FEATURE_COLS if c in df_features.columns]

print(" Features selected: {}".format(len(FEATURE_COLS)))
print("   {}".format(FEATURE_COLS))


##  BƯỚC 8: XÂY DỰNG MÔ HÌNH DỰ ĐOÁN

### Các mô hình đề xuất:

| Model | Ưu điểm | Nhược điểm | Khi nào dùng |
|-------|---------|------------|--------------|
| **LightGBM/XGBoost** | Nhanh, xử lý tốt tabular data, feature importance | Không capture time dependency tốt | **Khuyến nghị chính** |
| **SARIMA/Prophet** | Capture seasonality tốt | Chỉ fit 1 series, chậm với nhiều destinations | Phân tích từng địa điểm |
| **Random Forest** | Robust, ít tuning | Chậm hơn gradient boosting | Baseline |
| **Neural Network** | Flexible | Cần nhiều data, khó interpret | Khi có nhiều dữ liệu |

###  Chiến lược đề xuất:
1. **Baseline**: Naive forecast (dùng cùng tháng năm trước)
2. **Main Model**: LightGBM với all features
3. **Ensemble**: Kết hợp LightGBM + Prophet cho top destinations

In [ ]:
# =============================================================================
# BƯỚC 8.1: BASELINE MODEL (Seasonal Naive)
# =============================================================================

def seasonal_naive_forecast(df, forecast_horizon=1):
    """
    Baseline: Dùng traffic của cùng tháng năm trước
    """
    df = df.copy()
    df['prediction_naive'] = df['lag_12m']  # Cùng tháng năm trước
    return df

# Apply baseline
val_df_baseline = seasonal_naive_forecast(val_df)
val_df_baseline = val_df_baseline.dropna(subset=['prediction_naive', 'traffic'])

# Đánh giá baseline
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae_baseline = mean_absolute_error(val_df_baseline['traffic'], val_df_baseline['prediction_naive'])
rmse_baseline = np.sqrt(mean_squared_error(val_df_baseline['traffic'], val_df_baseline['prediction_naive']))
mape_baseline = np.mean(np.abs((val_df_baseline['traffic'] - val_df_baseline['prediction_naive']) / 
                               (val_df_baseline['traffic'] + 1))) * 100

print(" BASELINE (Seasonal Naive) Performance on Validation Set:")
print("   MAE:  {}".format(mae_baseline))
print("   RMSE: {}".format(rmse_baseline))
print("   MAPE: {}%".format(mape_baseline))


In [ ]:
# =============================================================================
# BƯỚC 8.2: LIGHTGBM MODEL
# =============================================================================

# Install if needed
# !pip install lightgbm

import lightgbm as lgb

# Prepare data
X_train = train_df[FEATURE_COLS].fillna(0)
y_train = train_df[TARGET]
X_val = val_df[FEATURE_COLS].fillna(0)
y_val = val_df[TARGET]

# LightGBM parameters
params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'n_estimators': 500,
    'early_stopping_rounds': 50
}

print(" Training LightGBM model...")
model_lgb = lgb.LGBMRegressor(**params)
model_lgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='mae'
)

# Predict
y_pred_lgb = model_lgb.predict(X_val)

# Evaluate
mae_lgb = mean_absolute_error(y_val, y_pred_lgb)
rmse_lgb = np.sqrt(mean_squared_error(y_val, y_pred_lgb))
mape_lgb = np.mean(np.abs((y_val - y_pred_lgb) / (y_val + 1))) * 100

print("LIGHTGBM Performance on Validation Set:")
print("   MAE:  {} (Baseline: {}, Improvement: {}%)".format(mae_lgb, mae_baseline, (1 - mae_lgb / mae_baseline) * 100))
print("   RMSE: {} (Baseline: {})".format(rmse_lgb, rmse_baseline))
print("   MAPE: {}% (Baseline: {}%)".format(mape_lgb, mape_baseline))


In [ ]:
# =============================================================================
# BƯỚC 8.3: FEATURE IMPORTANCE ANALYSIS
# =============================================================================

# Get feature importance
importance_df = pd.DataFrame({
    'feature': FEATURE_COLS,
    'importance': model_lgb.feature_importances_
}).sort_values('importance', ascending=False)

# Visualize
fig, ax = plt.subplots(figsize=(10, 8))
top_n = 20
sns.barplot(data=importance_df.head(top_n), x='importance', y='feature', palette='viridis', ax=ax)
ax.set_title(f'Top {top_n} Feature Importance')
ax.set_xlabel('Importance')
plt.tight_layout()
plt.show()

print("\n Top 10 Most Important Features:")
print(importance_df.head(10).to_string(index=False))

##  BƯỚC 9: XÁC ĐỊNH ĐIỂM ĐẾN CÓ TRAFFIC CAO TRONG TƯƠNG LAI

### Mục tiêu chính: Dự đoán Top-K địa điểm có traffic cao nhất trong 1-3 tháng tới

**Hai cách tiếp cận:**
1. **Regression**: Dự đoán giá trị traffic → Ranking theo giá trị dự đoán
2. **Classification**: Phân loại Top-K vs Rest → Binary classification

**Metrics đánh giá:**
- **Precision@K**: Bao nhiêu % trong Top-K dự đoán thực sự nằm trong Top-K thực tế
- **Recall@K**: Bao nhiêu % của Top-K thực tế được capture bởi Top-K dự đoán
- **NDCG**: Đánh giá chất lượng ranking

In [ ]:
# =============================================================================
# BƯỚC 9.1: DỰ ĐOÁN VÀ RANKING TOP DESTINATIONS
# =============================================================================

def predict_and_rank(model, df, feature_cols, top_k=50):
    """
    Dự đoán traffic và ranking top-K destinations cho mỗi tháng
    """
    df = df.copy()
    X = df[feature_cols].fillna(0)
    df['predicted_traffic'] = model.predict(X)
    
    # Rank within each month
    df['rank_predicted'] = df.groupby('date_parsed')['predicted_traffic'].rank(ascending=False)
    df['rank_actual'] = df.groupby('date_parsed')['traffic'].rank(ascending=False)
    
    return df

# Apply to validation set
val_ranked = predict_and_rank(model_lgb, val_df, FEATURE_COLS, top_k=50)

# Tính Precision@K và Recall@K
def calculate_ranking_metrics(df, k=50):
    """
    Tính precision@K, recall@K cho mỗi tháng
    """
    results = []
    for date in df['date_parsed'].unique():
        month_df = df[df['date_parsed'] == date]
        
        top_k_predicted = set(month_df[month_df['rank_predicted'] <= k]['destination'])
        top_k_actual = set(month_df[month_df['rank_actual'] <= k]['destination'])
        
        if len(top_k_predicted) == 0 or len(top_k_actual) == 0:
            continue
            
        overlap = len(top_k_predicted & top_k_actual)
        precision = overlap / len(top_k_predicted)
        recall = overlap / len(top_k_actual)
        
        results.append({
            'date': date,
            'precision@k': precision,
            'recall@k': recall
        })
    
    return pd.DataFrame(results)

# Evaluate ranking
for k in [20, 50, 100]:
    ranking_metrics = calculate_ranking_metrics(val_ranked, k=k)
    avg_precision = ranking_metrics['precision@k'].mean()
    avg_recall = ranking_metrics['recall@k'].mean()
    print(" K={}: Precision@K={}, Recall@K={}".format(k, avg_precision, avg_recall))


In [ ]:
# =============================================================================
# BƯỚC 9.2: DỰ BÁO CHO CÁC THÁNG SẮP TỚI (PRODUCTION)
# =============================================================================

def forecast_upcoming_months(model, df, feature_cols, n_months=3, top_k=30):
    """
    Dự đoán top-K destinations cho n_months tháng tới
    
    Cách hoạt động:
    1. Lấy dữ liệu của tháng hiện tại
    2. Tạo features cho tháng tiếp theo
    3. Dự đoán traffic
    4. Ranking và trả về top-K
    """
    
    # Lấy tháng cuối cùng trong dataset
    latest_date = df['date_parsed'].max()
    latest_month = latest_date.month
    latest_year = latest_date.year
    
    print(" Dữ liệu mới nhất: {}".format(latest_date.strftime('%Y-%m')))
    print(" Dự báo cho {} tháng tới:".format(n_months))
    
    results = []
    
    for i in range(1, n_months + 1):
        # Tính tháng cần dự báo
        forecast_month = (latest_month + i - 1) % 12 + 1
        forecast_year = latest_year + (latest_month + i - 1) // 12
        
        print(" Tháng {}/{}:".format(forecast_month, forecast_year))
        
        # Lấy dữ liệu của tháng này từ các năm trước để tạo features
        # (trong thực tế sẽ cần update lag features phức tạp hơn)
        month_data = df[df['month'] == forecast_month].copy()
        
        if len(month_data) == 0:
            continue
        
        # Dự đoán
        X = month_data[feature_cols].fillna(0)
        month_data['predicted_traffic'] = model.predict(X)
        
        # Ranking
        month_data['rank'] = month_data['predicted_traffic'].rank(ascending=False)
        top_destinations = month_data.nlargest(top_k, 'predicted_traffic')[
            ['destination', 'province', 'predicted_traffic', 'has_strong_seasonality', 'is_peak_month']
        ].reset_index(drop=True)
        
        top_destinations['forecast_month'] = forecast_month
        top_destinations['forecast_year'] = forecast_year
        results.append(top_destinations)
        
        print("   Top 5: {}".format(', '.join(top_destinations['destination'].head(5).tolist())))
        print()
    
    return pd.concat(results, ignore_index=True) if results else pd.DataFrame()

# Chạy forecast
forecast_results = forecast_upcoming_months(model_lgb, df_features, FEATURE_COLS, n_months=3, top_k=30)
print("Đã dự báo {} recommendations".format(len(forecast_results)))


In [ ]:
# =============================================================================
# BƯỚC 9.3: HIỂN THỊ KẾT QUẢ DỰ BÁO
# =============================================================================

# Lấy tháng dự báo
if len(forecast_results) > 0:
    forecast_results['month_year'] = forecast_results.apply(
        lambda x: f"Tháng {int(x['forecast_month'])}/{int(x['forecast_year'])}", axis=1
    )
    
    # Pivot để hiển thị top destinations mỗi tháng
    print("="*80)
    print(" TOP ĐIỂM ĐẾN DỰ BÁO CÓ TRAFFIC CAO NHẤT")
    print("="*80)
    
    for month_year in forecast_results['month_year'].unique():
        month_data = forecast_results[forecast_results['month_year'] == month_year].head(15)
        
        print("{}:".format(month_year))
        print("-"*60)
        for i, row in month_data.iterrows():
            peak_flag = "" if row['is_peak_month'] == 1 else ""
            seasonal_flag = "" if row['has_strong_seasonality'] else ""
            print("   {}. {} ({}) {}{}".format(month_data.index.get_loc(i) + 1, row['destination'], row['province'], peak_flag, seasonal_flag))
            print("       Predicted traffic: {}".format(row['predicted_traffic']))
        print()
else:
    print("️ Không có kết quả dự báo")

---
##  BƯỚC 10: TÓM TẮT VÀ ĐỀ XUẤT TIẾP THEO

###  Đã hoàn thành:
1. **Data Exploration**: Phân tích 174,060 records, 967 destinations, 15 năm dữ liệu
2. **Feature Engineering**: Tạo 35+ features từ lag, rolling, seasonality, weather, infrastructure
3. **Modeling**: LightGBM với early stopping, đạt cải thiện MAE so với baseline
4. **Prediction**: Dự báo top destinations cho 1-3 tháng tới
5. **Ranking Evaluation**: Precision@K, Recall@K metrics

###  Cải tiến tiếp theo:

| Ưu tiên | Task | Chi tiết |
|---------|------|----------|
|  **Cao** | Hyperparameter tuning | GridSearch/Optuna cho LightGBM |
|  **Cao** | Cross-validation | TimeSeriesSplit với multiple folds |
|  **Trung bình** | Thêm features | Lễ hội (festival calendar), events đặc biệt |
|  **Trung bình** | Prophet model | Cho top destinations có strong seasonality |
|  **Thấp** | Neural Network | LSTM/Transformer cho sequence modeling |
|  **Thấp** | API/Dashboard | Productionize predictions |

###  Insights quan trọng:
1. **Lag features** (đặc biệt lag_12m - cùng tháng năm trước) là quan trọng nhất
2. **Destinations có strong seasonality** dễ dự đoán hơn
3. **Weather và infrastructure** có ảnh hưởng nhưng không mạnh bằng historical patterns
4. **Zero-inflation** cần được xử lý đặc biệt (nhiều destinations có 0 traffic)

In [ ]:
# =============================================================================
# BONUS: EXPORT KẾT QUẢ
# =============================================================================

# Save trained model
import joblib

# Lưu model
joblib.dump(model_lgb, '../models/lgb_traffic_predictor.pkl')
print(" Model saved to ../models/lgb_traffic_predictor.pkl")

# Lưu predictions
if len(forecast_results) > 0:
    forecast_results.to_csv('../data/predictions/upcoming_top_destinations.csv', index=False)
    print(" Predictions saved to ../data/predictions/upcoming_top_destinations.csv")

# Lưu feature importance
importance_df.to_csv('../data/predictions/feature_importance.csv', index=False)
print(" Feature importance saved to ../data/predictions/feature_importance.csv")

print("\n HOÀN THÀNH PHÂN TÍCH!")